## Import Libraries and Load Data

In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline
from nltk import word_tokenize
import warnings;
warnings.filterwarnings('ignore')
from collections import Counter 

In [2]:
df_raw = pd.read_excel('tnved-CIS-02.xls')

In [3]:
df_raw.head()

,TNVED,Cust_ID,Name,Unit,FULL_TEXT
0,0101,6116.0,"Лошади, ослы, мулы и лошаки живые:",NaN,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ:"
1,0101210000,5129.0,––чистопородные племенные животные,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ЧИ..."
2,010129,6730.0,––прочие:,NaN,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ПР..."
3,0101291000,6689.0,–––убойные,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ПР..."
4,0101299000,5596.0,–––прочие,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ПР..."


In [4]:
df_raw.shape

(15927, 5)

The dataset contains 541,909 transactions. That is a pretty good number for us.

## Treat Missing Data

In [5]:
# check for missing values
df_raw.isnull().sum()

TNVED           0
Cust_ID       135
Name           31
Unit         4375
FULL_TEXT      31
dtype: int64

<br>
Since we have sufficient data, we will drop all the rows with missing values.

In [6]:
# remove missing values
df_raw.dropna(inplace=True)

# again check missing values
df_raw.isnull().sum()

TNVED        0
Cust_ID      0
Name         0
Unit         0
FULL_TEXT    0
dtype: int64

## Data Preparation

Let's convert the StockCode to string datatype.

In [7]:
df_raw['TNVED']= df_raw['TNVED'].astype(str)

Let's check out the number of unique customers in our dataset.

In [8]:
customers = df_raw["Cust_ID"].unique().tolist()
len(customers)

1996

In [9]:
import string
import nltk
import nltk
nltk.download('stopwords')
lst_stopwords = nltk.corpus.stopwords.words('russian')
lst_stopwords.append('прочие')
lst_stopwords.append('включая')
import re
lst_stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jora\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [10]:
def clean_text(text, tokenizer, stopwords):
    """Pre-process text and generate tokens

    Args:
        text: Text to tokenize.

    Returns:
        Tokenized text.
    """
    text = str(text).lower()  # Lowercase words
    text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Remove punctuation

    tokens = tokenizer(text)  # Get tokens from text
    tokens = [t for t in tokens if not t in lst_stopwords]  # Remove stopwords
    tokens = ["" if t.isdigit() else t for t in tokens]  # Remove digits
    tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
    return tokens

In [11]:
text_columns = ['FULL_TEXT']

df = df_raw
df[ "FULL_TEXT"] = df[ "FULL_TEXT"].fillna(" ")

for col in text_columns:
    df[col] = df[col].astype(str)

# Create text column based on title, description, and content
df["text"] = df[text_columns].apply(lambda x: " | ".join(x), axis=1)
df["tokens"] = df["text"].map(lambda x: clean_text(x, word_tokenize, lst_stopwords))

#Remove duplicated after preprocessing
_, idx = np.unique(df["tokens"], return_index=True)
df = df.iloc[idx, :]

# Remove empty values
df = df.loc[df.tokens.map(lambda x: len(x) > 0), ["text", "tokens"]]

print(f"Original dataframe: {df_raw.shape}")
print(f"Pre-processed dataframe: {df.shape}")

Original dataframe: (11454, 7)
Pre-processed dataframe: (10686, 2)


In [12]:
df_raw

,TNVED,Cust_ID,Name,Unit,FULL_TEXT,text,tokens
1,0101210000,5129.0,––чистопородные племенные животные,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ЧИ...","ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ЧИ...","[лошади, ослы, мулы, лошаки, живые, лошади, чи..."
3,0101291000,6689.0,–––убойные,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ПР...","ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ПР...","[лошади, ослы, мулы, лошаки, живые, лошади, уб..."
4,0101299000,5596.0,–––прочие,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ПР...","ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ПР...","[лошади, ослы, мулы, лошаки, живые, лошади]"
5,0101300000,5117.0,–ослы,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ОСЛЫ","ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ОСЛЫ","[лошади, ослы, мулы, лошаки, живые, ослы]"
6,0101900000,5686.0,–прочие,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ПРОЧИЕ","ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ПРОЧИЕ","[лошади, ослы, мулы, лошаки, живые]"
...,...,...,...,...,...,...,...
15785,9506620000,5694.0,––мячи надувные,шт,ИНВЕНТАРЬ И ОБОРУДОВАНИЕ ДЛЯ ЗАНЯТИЙ ОБЩЕЙ ФИ...,ИНВЕНТАРЬ И ОБОРУДОВАНИЕ ДЛЯ ЗАНЯТИЙ ОБЩЕЙ ФИ...,"[инвентарь, оборудование, занятий, общей, физк..."
15787,9506691000,6802.0,–––мячи для крикета и поло,шт,ИНВЕНТАРЬ И ОБОРУДОВАНИЕ ДЛЯ ЗАНЯТИЙ ОБЩЕЙ ФИ...,ИНВЕНТАРЬ И ОБОРУДОВАНИЕ ДЛЯ ЗАНЯТИЙ ОБЩЕЙ ФИ...,"[инвентарь, оборудование, занятий, общей, физк..."
15788,9506699000,5291.0,–––прочие,шт,ИНВЕНТАРЬ И ОБОРУДОВАНИЕ ДЛЯ ЗАНЯТИЙ ОБЩЕЙ ФИ...,ИНВЕНТАРЬ И ОБОРУДОВАНИЕ ДЛЯ ЗАНЯТИЙ ОБЩЕЙ ФИ...,"[инвентарь, оборудование, занятий, общей, физк..."
15790,9506701000,5769.0,––ледовые коньки,пар,ИНВЕНТАРЬ И ОБОРУДОВАНИЕ ДЛЯ ЗАНЯТИЙ ОБЩЕЙ ФИ...,ИНВЕНТАРЬ И ОБОРУДОВАНИЕ ДЛЯ ЗАНЯТИЙ ОБЩЕЙ ФИ...,"[инвентарь, оборудование, занятий, общей, физк..."


There are 4,372 customers in our dataset. For each of these customers we will extract their buying history. In other words, we can have 4,372 sequences of purchases.

It is a good practice to set aside a small part of the dataset for validation purpose. Therefore, I will use data of 90% of the customers to create word2vec embeddings. Let's split the data.

In [13]:
# shuffle customer ID's
random.shuffle(customers)

# extract 90% of customer ID's
customers_train = [customers[i] for i in range(round(0.9*len(customers)))]

# split data into train and validation set
train_df = df_raw[df_raw['Cust_ID'].isin(customers_train)]
validation_df = df_raw[~df_raw['Cust_ID'].isin(customers_train)]

Let's create sequences of purchases made by the customers in the dataset for both the train and validation set.

In [14]:
# list to capture purchase history of the customers
purchases_train = []

# populate the list with the product codes
for i in tqdm(customers_train):
    temp = train_df[train_df["Cust_ID"] == i]["TNVED"].tolist()
    purchases_train.append(temp)

100%|████████████████████████████████████████████████████████████████████████████| 1796/1796 [00:00<00:00, 5189.58it/s]


In [15]:
# list to capture purchase history of the customers
purchases_val = []

# populate the list with the product codes
for i in tqdm(validation_df['Cust_ID'].unique()):
    temp = validation_df[validation_df["Cust_ID"] == i]["TNVED"].tolist()
    purchases_val.append(temp)

100%|██████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 5262.05it/s]


In [16]:
docs = df_raw["tokens"].values
tokenized_docs = df_raw["tokens"].values
vocab = Counter()
for token in tokenized_docs:
    vocab.update(token)

## Build word2vec Embeddings for Products

In [17]:
# train word2vec model
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

In [18]:
model.build_vocab(tokenized_docs)

In [19]:
model.train(tokenized_docs, total_examples=model.corpus_count, epochs=30, report_delay=1)

(6878222, 7517850)

In [20]:
# save word2vec model
model.save("word2vec_2.model")

As we do not plan to train the model any further, we are calling init_sims(), which will make the model much more memory-efficient.

In [21]:
model.init_sims(replace=True)

In [22]:
print(model)

Word2Vec(vocab=5634, size=100, alpha=0.03)


Now we will extract the vectors of all the words in our vocabulary and store it in one place for easy access.

In [23]:
# extract all vectors
X = model[model.wv.vocab]

X.shape

(5634, 100)

In [25]:
model.wv.similar_by_vector("свинина")

[('грудинки', 0.9217213988304138),
 ('стрики', 0.9154728651046753),
 ('корейки', 0.9049151539802551),
 ('окорока', 0.8635772466659546),
 ('края', 0.8194581270217896),
 ('лопатки', 0.8155902624130249),
 ('баранина', 0.8007454872131348),
 ('козлятина', 0.7877663373947144),
 ('пищевую', 0.7829128503799438),
 ('пищевая', 0.765634298324585)]

## Start Recommending Products

In [27]:
products = train_df[["TNVED", "FULL_TEXT"]]

# remove duplicates
products.drop_duplicates(inplace=True, subset='TNVED', keep="last")

# create product-ID and product-description dictionary
products_dict = products.groupby('TNVED')['FULL_TEXT'].apply(list).to_dict()

In [28]:
# test the dictionary
products_dict['0101210000']

[' ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ЧИСТОПОРОДНЫЕ ПЛЕМЕННЫЕ ЖИВОТНЫЕ']

In [34]:
def similar_products(v, n = 6):
    
    # extract most similar products for the input vector
    ms = model.similar_by_vector(v, topn= n+1)[1:]
    
    # extract name and similarity score of the similar products
    new_ms = []
    for j in ms:
        pair = (products_dict[j[0]][0], j[1])
        new_ms.append(pair)
        
    return new_ms        

In [40]:
similar_products(model.similar_by_word('ЛОШАДИ'))

KeyError: "word 'ЛОШАДИ' not in vocabulary"